# Clustering on bernoulli embeddings model with dynamic context

*Yufan Zhuang*<br>
*Xuewei Du*<br>
*Columbia University*

## Introduction

Word embeddings are powerful in terms of learning latent semantic structure in language.  Recently, Rudolph and Blei (Rudolph and Blei, 2018) explained dynamic embeddings for language evolution, in which he used exponential family embeddings (Rudolph et al., 2018) which can change over time to capture the change in meanings of words over time. Here in this report, we would like to elaborate some findings on modification of the previous models, where we will have context vectors that change over time, and we will have Gaussian Mixture Model(GMM) to cluster words. We study how this new model performs on United Nations general debates data. 

### Background

#### Word embeddings

Word embeddings can be used to represent text data. For instance, in frequency based embedding, if we do one-hot encoding on a particular text data, then we will have a vector of the same length as the number of unique words to represent each word. To represent a particular word, we can set the corresponding position in the vector to 1, and others to 0. However, more generally, we do not necessarily have to use such high dimensional vectors to represent words. We can have one vector of much lower dimension(i.e. a much shorter vector) to represent a word, and each position in that vector can be decimal numbers instead of (0,1). In probabilistic programming setting, each position in this embedding vector can be viewed as a parameter. 

#### Context vector

Each word in the observed text is not only dependent upon word embeddings but also its context. This means each word is related to a few words before and after itself. As Rudolph and Blei (Rudolph and Blei, 2018) pointed out in the paper, the typical context has a size between 2 and 10, and each word is modeled conditionally on the words before and after. 

#### Prior Work

Rudolph and Blei (Rudolph and Blei, 2018) explained how dynamic embeddings work. In particular, for each time slice, there are some observed words. Each observed word is dependent upon both its embeddings under the same time slice and its context vector which is shared among all time slices. Over time, the embeddings change while the context stays the same. 

## Model

We are proposing clustering using GMM Bernoulli embeddings and dynamic context vectors. There are practically three components to the model, Bernoulli, dynamic and GMM. <br>

Bernoulli means the Bernoulli embedding model(Rudolph and Blei, 2018). Under the Bernoulli embedding model, we have a one-hot encoded embedding vector of length V to represent each word. The corresponding position is 1 for a particular word, and 0 for other positions.  Each position in the embedding vector of each observed word is modeled using Bernoulli distribution with probability $p_{iv}$ for each $v$ in the entire vocabulary $V$. $$P(x_{iv}|x_{c_i}) = Bern(p_{iv})$$. The log odds $\eta_{iv} = log \frac{p_{iv}}{1-p_{iv}}$, which is a function of $p_{iv}$, can be specified as $$\eta_{iv} = \rho_v^T(\sum_{j \in c_i}\sum_{v'}\alpha_{v'}x_{jv'})$$ Here $\rho_v \in R^K$ represents embeddings and $\alpha_{v'} \in R^K$ represents context. The value K is set in advance. Therefore the log-odds is an inner-product between the embeddings and context vectors. 

Dynamic means dynamic context vector. In particular, the log odds can be specified as the inner-product between the embeddings and time-specific context vector. $$\eta_{iv} = \rho_v^T(\sum_{j \in c_i}\sum_{v'}\alpha_{v'}^{(t_i)}x_{jv'})$$ where $t_i$ is the time slice for the word $x_{iv}$.

GMM means Gaussian mixture model. We build a GMM on the embedding vectors $\rho_v$. $$P(\rho_v | \pi, \mu, \Lambda) = \Pi_{k=1}^{K}(\pi_k N(\rho_v | \mu_k, \Lambda_k))$$ The goal is to learn parameters $\pi$, $\mu$ and $\Lambda$. Their prior can be set as $$P(\pi) \sim Dir(\alpha)$$ $$P(\mu_k) \sim N(0, I)$$ $$P(\Lambda_k) \sim Wishart(V, n)$$.

## Data

The data we use is regarding United Nations general debates (Kaggle.com, 2018). It has transcripts of general debates from 1970 to 2016. Each row in the data represents a speech given by a particular country’s representative in a particular year and session. The raw data has four columns: session, year, country and text. Session means the UN session and there is one session per year. Country means the country which the representative is from. Text means the transcript of the representative’s speech. <br>

Preprocessing: Since the data is in years, it is natural to set each time slice to be a year. All the speeches in the same year are attached together. In the resulting data folder, we have one txt file for each year, and each txt file includes all the speeches in that year. 

## Inference

We offer two inference methods for the clustering, Hamiltonian Monte Carlo (HMC) and Stochastic Gradient Descent Variational Inference (SGDVI) to get the Bayes estimate. For the embedding model, we employ stochastic gradient descent to get the MAP estimate.

### Hamiltonian Monte Carlo

<img src="hmc_ill.png">
(Illustration from Wang et al., 2019)

Hamiltonian Monte Carlo was proposed in the late 80s as a method to investigate the structure of particles (Duane et al., 1987). It then was borrowed into the statistics community as a very efficient method to approximate the posterior. The basic idea is that instead of moving randomly as we did in MH, we move around a vector field that we have some directions to follow to get to unexplored regions. We utilized the tensorflow probability to implement this as the following.

### Stochastic Gradient Descent Variational Inference

Due to our setting, we can make the conjugate exponential family (CEF) argument such that under the mean-field assumption, the variational distributions will be in the same form as the distributions that they are trying to approximate.

$$
\begin{align*}
p(\pi,\mu,\Lambda, c|\rho) \approx q(\pi)[\prod_{j=1}^{K} q(\mu_j)q(\Lambda_j)][\prod_{i=1}^{n} q(c_i)]
\end{align*}$$

$$
\begin{align*}
q(\pi) &= Dir(\alpha_1', \alpha_2',...,\alpha_k') \\
q(\mu_j) &= N(m_j',\Sigma_j') \\
q(\Lambda_j) &= Wishart(a_j',B_j')  \\
q(c_i) &= Discrete(\phi_i) 
\end{align*}
$$

The ELBO is then
$$
L = E_q[\ln p(\rho,\pi,\mu,\Lambda, c) - \ln q ] + const.
$$

Analytical optimal update schemes can be derived, but since we are dealing with high-dimensional big data, we choose to use SGD methods instead. 

The two methods are both implemented as classes.

## Criticizm

Since we built an embedding model and a topic model at the same time, we will evaluate it in two metrics, the log-likelihood and the topic coherence. 

For the log-likelihood, 

## Reference

Wang, Z., M. Broccardo, and J. Song* (2019). Hamiltonian Monte Carlo methods for subset simulation in reliability analysis. Structural Safety. Vol. 76, 51-67

Rudolph, M. and Blei, D. (2018). Dynamic Bernoulli Embeddings for Language Evolution. [online] Arxiv.org. Available at: https://arxiv.org/abs/1703.08052 [Accessed 27 Nov. 2018].

Rudolph, M., Ruiz, F., Mandt, S. and Blei, D. (2018). Exponential Family Embeddings. [online] Arxiv.org. Available at: https://arxiv.org/abs/1608.00778 [Accessed 27 Nov. 2018].

Kaggle.com. (2018). UN General Debates. [online] Available at: https://www.kaggle.com/unitednations/un-general-debates [Accessed 27 Nov. 2018].

Duane, S., Kennedy, A. D., Pendleton, B. J., & Roweth, D. (1987). Hybrid monte carlo. Physics letters B, 195(2), 216-222.